<a href="https://colab.research.google.com/github/shahnupur1901/Filtrone-App/blob/master/DeepLearningModels/ModelForArticleType.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download paramaggarwal/fashion-product-images-small
! unzip /content/fashion-product-images-small.zip

In [ ]:
import numpy as np
import pandas as pd
import os
path = "/content/myntradataset"
csv_path = "/content/myntradataset/styles.csv"
img_path = "/content/myntradataset/images"
df = pd.read_csv(csv_path, error_bad_lines=False, warn_bad_lines=False)

df = df[df['baseColour'].notna()]
df['id'] = df['id'].apply(lambda x: str(x)+'.jpg')
df = df[df['id'].isin(os.listdir(img_path))]
df = df.reset_index(drop=True)
df = df.groupby(['articleType']).filter(lambda x: len(x) >= 20)
print('DF shape:', df.shape)

DF shape: (44005, 10)


In [ ]:
df = df[df.articleType != "Lip Gloss"]
df = df[df.articleType != "Lipstick"]
df = df[df.articleType != "Nail Polish"]
df = df[df.articleType != "Scarves"]
df = df[df.articleType != "Wallets"]
df = df[df.articleType != "Muffler"]
df = df[df.articleType != 'Face Wash and Cleanser']
df = df[df.articleType != 'Foundation and Primer']
df = df[df.articleType != "Highlighter and Blush"]
df = df[df.articleType != "Face Moisturisers"]
df = df[df.articleType != "Compact"]
df = df[df.articleType != "Lip Liner"]
df = df[df.articleType != "Eyeshadow"]
df = df[df.articleType != "Stoles"]
df = df[df.articleType != "Cufflinks"]
df = df[df.articleType != "Sunscreen"]
df = df[df.articleType != "Dresses"]
df = df[df.articleType != "Kurtis"]
df = df[df.articleType != "Shirts"]
df = df[df.articleType != "Sweaters"]
df = df[df.articleType != "Sweatshirts"]
df = df[df.articleType != "Tops"]
df = df[df.articleType != "Tshirts"]
df = df[df.articleType != "Kajal and Eyeliner"]

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df,test_size = 0.1, random_state=42)
train_df, validation_df = train_test_split(df,test_size = 0.2, random_state=57)

In [ ]:
df["articleType"].unique()

array(['Jeans', 'Watches', 'Track Pants', 'Socks', 'Casual Shoes',
       'Belts', 'Flip Flops', 'Handbags', 'Bra', 'Sandals',
       'Shoe Accessories', 'Deodorant', 'Formal Shoes', 'Bracelet',
       'Flats', 'Kurtas', 'Sports Shoes', 'Shorts', 'Briefs', 'Sarees',
       'Perfume and Body Mist', 'Heels', 'Sunglasses', 'Innerwear Vests',
       'Pendant', 'Laptop Bag', 'Night suits', 'Skirts', 'Ring',
       'Kurta Sets', 'Clutches', 'Backpacks', 'Caps', 'Trousers',
       'Earrings', 'Camisoles', 'Boxers', 'Jewellery Set', 'Dupatta',
       'Capris', 'Bath Robe', 'Mufflers', 'Tunics', 'Jackets', 'Trunk',
       'Lounge Pants', 'Necklace and Chains', 'Duffel Bag',
       'Sports Sandals', 'Free Gifts', 'Tracksuits', 'Fragrance Gift Set',
       'Bangle', 'Nightdress', 'Ties', 'Leggings', 'Mobile Pouch',
       'Messenger Bag', 'Accessory Gift Set', 'Suspenders', 'Patiala',
       'Stockings', 'Churidar', 'Lounge Shorts', 'Gloves', 'Salwar',
       'Jeggings'], dtype=object)

In [ ]:

batch_size = 16
df.sample(frac=1, axis=1)
from keras_preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(
    rescale= 1./255,
    shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    #validation_split=0.2
)
validation_image_generator = ImageDataGenerator(
    rescale= 1./255,
    #validation_split=0.2
)

training_generator = train_image_generator.flow_from_dataframe(
    dataframe=train_df,
    directory=img_path,
    x_col="id",
    y_col="articleType",
    target_size=(96,96),
    batch_size=batch_size,
    #subset="training"
)

validation_generator = validation_image_generator.flow_from_dataframe(
    dataframe=validation_df,
    directory=img_path,
    x_col="id",
    y_col="articleType",
    target_size=(96,96),
    batch_size=batch_size,
    #subset="validation"
)
test_generator = validation_image_generator.flow_from_dataframe(
    dataframe=test_df,
    directory=img_path,
    x_col="id",
    y_col="articleType",
    target_size=(96,96),
    batch_size=batch_size,
    #subset="validation"
)

classes = len(training_generator.class_indices)

Found 22544 validated image filenames belonging to 67 classes.
Found 5636 validated image filenames belonging to 67 classes.
Found 2818 validated image filenames belonging to 67 classes.


In [ ]:
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Dropout 
# create the base pre-trained model
base_model = MobileNetV2(input_shape=(96, 96, 3), include_top=False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

9420800/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 48, 48, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 48, 48, 32)   0           bn_Conv1[0][0]                   
_______________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
batch_size = 32
total_train = train_df.shape[0]
total_validation = validation_df.shape[0]
vgg_classifier = model.fit(training_generator,
steps_per_epoch=(total_train//batch_size),
epochs = 20,
validation_data=validation_generator,
validation_steps=(total_validation//batch_size),
batch_size = batch_size,
verbose = 1)

Epoch 1/20
704/704 [==============================] - 105s 149ms/step - loss: 0.7662 - accuracy: 0.7590 - val_loss: 0.5715 - val_accuracy: 0.8164
Epoch 2/20
704/704 [==============================] - 105s 149ms/step - loss: 0.7521 - accuracy: 0.7594 - val_loss: 0.5870 - val_accuracy: 0.8104
Epoch 3/20
704/704 [==============================] - 105s 149ms/step - loss: 0.7440 - accuracy: 0.7671 - val_loss: 0.5819 - val_accuracy: 0.8175
Epoch 4/20
704/704 [==============================] - 104s 148ms/step - loss: 0.7146 - accuracy: 0.7677 - val_loss: 0.5947 - val_accuracy: 0.8082
Epoch 5/20
704/704 [==============================] - 104s 148ms/step - loss: 0.6990 - accuracy: 0.7702 - val_loss: 0.5713 - val_accuracy: 0.8132
Epoch 6/20
704/704 [==============================] - 104s 148ms/step - loss: 0.6837 - accuracy: 0.7776 - val_loss: 0.5533 - val_accuracy: 0.8203
Epoch 7/20
704/704 [==============================] - 104s 148ms/step - loss: 0.6528 - accuracy: 0.7859 - val_loss: 0.5453 -

In [ ]:
model.save('modelForArticleType82.h5')
result = model.evaluate(test_generator,batch_size=batch_size)
print("test_loss, test accuracy",result)

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


177/177 [==============================] - 17s 96ms/step - loss: 0.4171 - accuracy: 0.8584
test_loss, test accuracy [0.4171394109725952, 0.8584102392196655]


In [ ]:
import json
with open('articleTypes.json', 'w') as fp:
    json.dump(training_generator.class_indices, fp)